# 1. Typo correction


## 1.1 Import the required libraries


In [2]:
from transformers import pipeline
from langdetect import detect_langs, DetectorFactory
from textblob import TextBlob
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

language_detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection") # this model is 1.1 gigabyte so it will take around 5 mins to download it
typo_corrector = pipeline("text2text-generation", model="oliverguhr/spelling-correction-english-base", max_length=1000)

DetectorFactory.seed = 0

## 1.2 Load in the raw data

### Base Data

In [3]:
# Determine the location of the dataframe containing the translated text
base_tilt_data_location = "../../data/example_data/output/base_data/base_flagged_products.csv"

# use raw_df
base_tilt_data = pd.read_csv(base_tilt_data_location)

# Display the dataframe
display(base_tilt_data)

,products_id,products_and_services,to_process
0,164399edbf8e880dc2e856f50d51e720bd0a8abe,"fish, frozen and deep-frozen",False
1,b0d3c55743b1b858ec2843c8870116bb8af543fd,drilling and test boring - equipment,False
2,b14c038972e6a52bfbf3ffbe77def57a62c5b9cf,well-management services,False
3,abadc2542b4b5c1ecfe41c22afb2347b1d9b65af,electronic data processing - software,False
4,60c58ad2ef34d96fae028f1039fab03dec9eb9a2,communication,False
...,...,...,...
32058,e8eeb99f7e34e92213e57b7282bc8ed2941df4c8,roberval balance and plates,True
32059,b2262cefc9feb523966ca1bfae6722ca658b2201,precision weights and masses,True
32060,9b4e98ccfa614c6744a3b964d8d29eeda598e7fe,casting and machining of cuprous alloys,True
32061,a56bfdd9971ddba76de33e5dd394faab63d2c58c,trading in non-ferrous products,True


### tilt Italy Data

In [4]:
# Determine the location of the dataframe containing the translated text
italy_tilt_data_location = "../../data/example_data/output/italy_data/italy_flagged_products.csv"

# use raw_df
italy_tilt_data = pd.read_csv(italy_tilt_data_location)

# Display the dataframe
display(italy_tilt_data)

,products_and_services,products_id,to_process
0,peas,a7849fec-bf97-48b0-b89b-cb910a8d6dc8,False
1,cauliflowers,e09840c2-df4a-4920-b91b-e23177b5f5b3,False
2,b2b services,616bd4f5-2977-4e43-a81b-accfd0e0db5f,False
3,fuel,2a782696-0317-43ad-8f2d-0560e17acd8d,False
4,financial brokerage,d9fca333-22ba-401e-a2bf-59792cf7a58f,False
...,...,...,...
3430,bookstore,0bb3c167-f281-404a-b16e-a190c8e9be4b,True
3431,pharmacy,a9a7cb96-77f7-4859-9157-9b7bbf64197b,True
3432,internal transport containers,5347fedc-9189-4140-9f98-4ac39f7e1e82,True
3433,food supplies for catering industry,6a296ca9-9f97-42d8-8fc6-188e575990dd,True


## 1.3 Apply typo correction module

### Helper functions

In [5]:
def conf_ld_detect_language(text, model="def"):
    """Language detection wrapper.
    
    Returns detected language (ISO-code) and confidence of detection. In case of 
    failure of detection string 'ident_fail' and a pd.NA value for confidence is 
    returned.
    
    Args:
        text (str): The string for which language shall be detected.
        model (str): The model to be used for language detection. Defaults to langdetect model.
    Returns:
        str: The detected language (ISO-code).
    """
    try:
        if model == "def":
            highest_conf = detect_langs(text)[0]
            return highest_conf.lang
        elif model == "huggingface":
            result = language_detector(text)[0]
            return str(result["label"])
    except:   
        return "ident_fail", pd.NA

In [6]:
def typo_correction(text="", model="def"):
    """Typo correction wrapper.
    
    Returns corrected text. In case of failure of correction the original text 
    is returned. 
    
    Args:
        text (str): The string to be corrected.
        model (str): The model to be used for typo correction. Defaults to textblob model.
    Returns:
        str: The corrected string.
    """
    try:
        if model == "def":
            return(TextBlob(text).correct().string)
        elif model == "huggingface":
            return(typo_corrector(text)[0]["generated_text"])
    except:
        return text

### Typo correction module

In [7]:
def typo_correct_df(df):
    """Typo correction wrapper for dataframes.
    
    Returns dataframe with corrected text. In case of failure of correction the 
    original text is returned. 
    
    Args:
        df (pd.DataFrame): The dataframe containing the text to be corrected.
    Returns:
        pd.DataFrame: The dataframe with corrected text.
    """
    # detect the language of the text but only for the rows that do not have a value in the automatic_processed_products_and_services column
    print("Detecting the language of the text...")
    # only take rows that have a True value in the to_process column
    to_process_df = df[df["to_process"] == True].copy()
    # exclude to_processed_df rows from df
    df = df[df["to_process"] == False].copy()
    to_process_df.loc[:, "language (ISO-code)"] = to_process_df["products_and_services"].progress_apply(lambda x: conf_ld_detect_language(x, model="huggingface"))

    # then take subset of english texts
    print("Taking subset of English texts...")
    english_df = to_process_df[to_process_df["language (ISO-code)"] == "en"]
    # exclude enlgish texts from the original df
    to_process_df = to_process_df[to_process_df["language (ISO-code)"] != "en"]

    # apply typo correction to english texts
    print("Applying typo correction...")
    english_df = english_df.copy()
    english_df.loc[:, "typo_corrected"] = english_df["products_and_services"].progress_apply(lambda x: typo_correction(x, model="huggingface"))

    # merge the corrected english texts with the original df
    print("Merging the corrected english texts with the original df...")
    df = pd.concat([to_process_df, english_df, df], ignore_index=True)
    # replace empty values in typo_corrected with the original text
    df["typo_corrected"].fillna(df["products_and_services"], inplace=True)
    # make typo_corrected lowercase and remove all dots at the end
    df["typo_corrected"] = df["typo_corrected"].str.lower().str.replace("\.$", "")
    return df

### Base Data

In [8]:
base_typo_corrected_df = typo_correct_df(base_tilt_data)

Detecting the language of the text...


  0%|          | 0/14378 [00:00<?, ?it/s]

100%|██████████| 14378/14378 [12:43<00:00, 18.82it/s]


Taking subset of English texts...
Applying typo correction...


100%|██████████| 11522/11522 [1:08:48<00:00,  2.79it/s]

Merging the corrected english texts with the original df...



C:\Users\selsabrouty\AppData\Local\Temp\ipykernel_5652\1874215916.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df["typo_corrected"] = df["typo_corrected"].str.lower().str.replace("\.$", "")


### tilt Italy Data

In [9]:
italy_typo_corrected_df = typo_correct_df(italy_tilt_data)

Detecting the language of the text...


100%|██████████| 838/838 [00:34<00:00, 24.04it/s]


Taking subset of English texts...
Applying typo correction...


100%|██████████| 609/609 [03:12<00:00,  3.17it/s]

Merging the corrected english texts with the original df...



C:\Users\selsabrouty\AppData\Local\Temp\ipykernel_5652\1874215916.py:37: FutureWarning: The default value of regex will change from True to False in a future version.
  df["typo_corrected"] = df["typo_corrected"].str.lower().str.replace("\.$", "")


## 1.4 Export the dataframe with the corrected text 

### Base Data

In [10]:
# Define the path for the new dataframe
output_path_base_typo_corrected = "../../data/example_data/output/base_data/base_typo_corrected_products.csv"

# Write the new dataframe to the path
base_typo_corrected_df.to_csv(output_path_base_typo_corrected, index=False)

### tilt Italy Data

In [11]:
# Define the path for the new dataframe
output_path_italy_typo_corrected = "../../data/example_data/output/italy_data/italy_typo_corrected_products.csv"

# Write the new dataframe to the path
italy_typo_corrected_df.to_csv(output_path_italy_typo_corrected, index=False)